In [ ]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras import datasets,models,layers

In [ ]:
# Adding TF Cifar10 Data ..
from keras.datasets import cifar10
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
print(X_train.shape)
X_train, Y_train = X_train[:10000,:,:,:], Y_train[:10000,:]

170498071/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)


In [ ]:
X_train, Y_train = X_train[:10000,:,:,:], Y_train[:10000,:]

In [ ]:
print(Y_train[-10:])
print(X_train.shape)
print(X_train[0,:,:,0])
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

[[9]
 [2]
 [2]
 [1]
 [6]
 [3]
 [9]
 [1]
 [1]
 [5]]
(10000, 32, 32, 3)
[[ 59  43  50 ... 158 152 148]
 [ 16   0  18 ... 123 119 122]
 [ 25  16  49 ... 118 120 109]
 ...
 [208 201 198 ... 160  56  53]
 [180 173 186 ... 184  97  83]
 [177 168 179 ... 216 151 123]]


In [ ]:
def normalize_img_one_hot(image, label):
  return image ,tf.one_hot(label, 10)

X_train, Y_train = normalize_img_one_hot(X_train,Y_train)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print(X_train.shape)
print(Y_train.shape)
Y_train = Y_train[:,0,:]


(10000, 32, 32, 3)
(10000, 1, 10)


In [ ]:
"""
ResNet-50
Reference:
[1] K. He et al. Deep Residual Learning for Image Recognition. CVPR, 2016
[2] K. He, X. Zhang, S. Ren, and J. Sun. Delving deep into rectifiers:
Surpassing human-level performance on imagenet classification. In
ICCV, 2015.
"""


from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
import tensorflow as tf


class BottleNeck(Model):
    """
    resnet bottleneeck block
    """
    def __init__(self, channels: int, down_sample=False):
      super().__init__()
      self.__channels = channels
      self.__down_sample = down_sample
      self.__strides = [2, 1] if down_sample else [1, 1]

      KERNEL_SIZE = (3, 3)
      INIT_SCHEME = "glorot_uniform"

      self.conv_1 = Conv2D(self.__channels/4, strides=self.__strides[0],
                            kernel_size=(1,1), padding="same", kernel_initializer=INIT_SCHEME, use_bias=False)
      self.bn_1 = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)
      self.conv_2 = Conv2D(self.__channels/4, strides=self.__strides[1],kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME, use_bias=False)
      self.bn_2 = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)
      self.conv_3 = Conv2D(self.__channels, strides=self.__strides[1],
                            kernel_size=(1,1), padding="same", kernel_initializer=INIT_SCHEME, use_bias=False)
      self.bn_3 = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)
      self.merge = Add()

      if self.__down_sample:
          # first block of each block cluster, image size decrease, channel increase
          self.res_conv = Conv2D(self.__channels, strides=self.__strides[0],
                            kernel_size=(1,1), padding="same", kernel_initializer=INIT_SCHEME, use_bias=False)
          self.res_bn = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)

    def call(self, inputs):
      res = inputs

      x = self.conv_1(inputs)
      x = self.bn_1(x)
      x = tf.nn.relu(x)
      x = self.conv_2(x)
      x = self.bn_2(x)
      x = tf.nn.relu(x)
      x = self.conv_3(x)
      x = self.bn_3(x)

      if self.__down_sample:
          # print(res.shape)
          res = self.res_conv(res)
          # print(res.shape)
          res = self.res_bn(res)

      # if not perform down sample, then add a shortcut directly
      x = self.merge([x, res])
      out = tf.nn.relu(x)

      return out


class ResNet50(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7),padding="same", kernel_initializer="glorot_uniform", use_bias=False)
        self.init_bn = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)
        # self.pool_2 = MaxPool2D(pool_size=(3, 3), strides=2, padding="same")

        # first half of stage 1
        self.conv_s1_1 = Conv2D(64, strides=1,kernel_size=(1,1), padding="same", kernel_initializer="glorot_uniform", use_bias=False)
        self.bn_s1_1 = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)
        self.conv_s1_2 = Conv2D(64, strides=1,kernel_size=(3,3), padding="same", kernel_initializer="glorot_uniform", use_bias=False)
        self.bn_s1_2 = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)
        self.conv_s1_3 = Conv2D(256, strides=1,kernel_size=(1,1), padding="same", kernel_initializer="glorot_uniform", use_bias=False)
        self.bn_s1_3 = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)
        self.side_conv_s1 = Conv2D(256, strides=1,kernel_size=(1,1), padding="same", kernel_initializer="glorot_uniform", use_bias=False)
        self.side_bn_s1 = BatchNormalization(axis=-1, momentum=0, epsilon=0.001)
        self.merge = Add()

        # stage 1
        self.res_1_2 = BottleNeck(256)
        self.res_1_3 = BottleNeck(256)

        # stage 2
        self.res_2_1 = BottleNeck(512, down_sample=True)
        self.res_2_2 = BottleNeck(512)
        self.res_2_3 = BottleNeck(512)

        # stage 3
        self.res_3_1 = BottleNeck(1024, down_sample = True)
        self.res_3_2 = BottleNeck(1024)
        self.res_3_3 = BottleNeck(1024)

        # stage 4
        self.res_4_1 = BottleNeck(2048, down_sample = True)
        self.res_4_2 = BottleNeck(2048)
        self.res_4_3 = BottleNeck(2048)
        # self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax", use_bias = False)

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        # out = self.pool_2(out)

        res = self.side_conv_s1(out)
        res = self.side_bn_s1(res)
        # first helf of stage 1
        out = self.conv_s1_1(out)
        out = self.bn_s1_1(out)
        out = tf.nn.relu(out)
        out = self.conv_s1_2(out)
        out = self.bn_s1_2(out)
        out = tf.nn.relu(out)
        out = self.conv_s1_3(out)
        out = self.bn_s1_3(out)
        out = self.merge([out,res])


        for res_block in [self.res_1_2, self.res_1_3, self.res_2_1, self.res_2_2, self.res_2_3, self.res_3_1, self.res_3_2, self.res_3_3, self.res_4_1, self.res_4_2, self.res_4_3]:
            out = res_block(out)
        # out = self.avg_pool(out)
        out = self.flat(out)
        # print(out.shape)
        out = self.fc(out)
        # print(out.shape)
        print("done out")
        return out

In [ ]:
model = ResNet50(10)
model.build(input_shape = (None,32,32,3))

#use categorical_crossentropy since the label is one-hot encoded
from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.1,momentum=0.9) #parameters suggested by He [1]
loss_fn2=tf.keras.losses.CategoricalCrossentropy()

model.compile(optimizer = opt, loss=loss_fn2, metrics=["accuracy"])
model.summary()

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
(None, 32, 32, 256)
(None, 16, 16, 512)
(None, 16, 16, 512)
(None, 8, 8, 1024)
(None, 8, 8, 1024)
(None, 4, 4, 2048)
(None, 32768)
(None, 10)
done out
Model: "res_net50_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_82 (Conv2D)          multiple                  9408      
                                                                 
 batch_normalization_82 (Bat  multiple                 256       
 chNormalization)                                                
                                                                 
 conv2d_83 (Conv2D)          multiple                  4096      
                                                                 
 batch_normalization_83 (Bat  multiple                 256       
 chNormalization)              

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(patience= 8, restore_best_weights=True, monitor="val_accuracy")
#I did not use cross validation, so the validate performance is not accurate.
STEPS = len(X_train) / 128
history = model.fit(X_train,Y_train, batch_size = 128, epochs=500, validation_data=(X_train, Y_train),callbacks=[es])